### Imports ###
The Qiskit job watcher magic string is useful, and surprisingly poorly documented


In [1]:
import time

import qiskitpool

from qiskit import IBMQ, QuantumCircuit
import qiskit.tools.jupyter

IBMQ.load_account()
provider = IBMQ.get_provider(group='open', project='main')

%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

## Single circuit trial

In [2]:
pool = qiskitpool.QPool(provider)
backend = provider.get_backend('simulator_statevector')

n_shots = 10

circ = QuantumCircuit(1, 1)
circ.x(0)
circ.measure(0, 0)

job = pool(circ, backend, shots=n_shots)

Print the pool to see what it's up to

In [3]:
print(pool)

ibmq_qasm_simulator:	[ ][ ][ ][ ][ ] : 0
ibmq_lima:	[ ][ ][ ][ ][ ] : 0
ibmq_belem:	[ ][ ][ ][ ][ ] : 0
ibmq_quito:	[ ][ ][ ][ ][ ] : 0
simulator_statevector:	[ ][ ][ ][ ][ ] : 1
simulator_mps:	[ ][ ][ ][ ][ ] : 0
simulator_extended_stabilizer:	[ ][ ][ ][ ][ ] : 0
simulator_stabilizer:	[ ][ ][ ][ ][ ] : 0
ibmq_manila:	[ ][ ][ ][ ][ ] : 0
ibm_nairobi:	[ ][ ][ ][ ][ ] : 0
ibm_oslo:	[ ][ ][ ][ ][ ] : 0



In [ ]:
while not job.poll():
    time.sleep(2)
    

assert(job.result().get_counts()['1'] == n_shots)

## 20 Circuits trial
You can increase this, but the IBMQ backend may throttle your job submissions

In [4]:
pool = qiskitpool.QPool(provider)
backend = provider.get_backend('simulator_statevector')

n_shots = 10
n_rounds = 20

circ = QuantumCircuit(1, 1)
circ.x(0)
circ.measure(0, 0)

# Create jobs
jobs = [pool(circ, backend, shots=n_shots) for i in range(n_rounds)]

### Print the pool while it is running to see what it's up to

In [26]:
print(pool)

ibmq_qasm_simulator:	[ ][ ][ ][ ][ ] : 0
ibmq_lima:	[ ][ ][ ][ ][ ] : 0
ibmq_belem:	[ ][ ][ ][ ][ ] : 0
ibmq_quito:	[ ][ ][ ][ ][ ] : 0
simulator_statevector:	[X][X][X][X][X] : 0
simulator_mps:	[ ][ ][ ][ ][ ] : 0
simulator_extended_stabilizer:	[ ][ ][ ][ ][ ] : 0
simulator_stabilizer:	[ ][ ][ ][ ][ ] : 0
ibmq_manila:	[ ][ ][ ][ ][ ] : 0
ibm_nairobi:	[ ][ ][ ][ ][ ] : 0
ibm_oslo:	[ ][ ][ ][ ][ ] : 0



Wait for the jobs to finish and then check the results

In [ ]:
# Wait for jobs to finish
n_complete = 0
while n_complete < n_rounds - 1:
    n_complete = 0
    for job in jobs:
        if job.poll():
            n_complete += 1

for job in jobs:
    assert(job.poll())
    assert(job.result().get_counts()['1'] == n_shots)

### Safely shut down the pool

In [ ]:
pool.join()